In [74]:
import dspy
import pandas as pd
import yaml
import os
from dotenv import load_dotenv
load_dotenv()

True

In [75]:
with open(os.getenv('PROMPT_CONFIG_FILE'), 'r') as f:
    prompt_config = yaml.safe_load(f)

In [76]:
df = pd.read_excel("jira_issues.xlsx")
question = df.iloc[3]["Question"]
summary = df.iloc[3]["Summary"]
quarter = df.iloc[3]["Quarter"]
data = df.values[:, 1:4]

In [77]:
gpt4 = dspy.OpenAI(model='gpt-4o', api_key=os.getenv('sk-proj-RC16O15SxLP3xQ1yApX2T3BlbkFJ0IziyB9mBKMaBBl2ebtB'))
dspy.configure(lm=gpt4)
# dspy.settings.configure(lm=gpt4, rm=colbertv2_wiki17_abstracts)

In [78]:
class GenerateSummary(dspy.Signature):
    question = dspy.InputField(desc="may contain relevant jira issue details")
    answer = dspy.OutputField(desc="quarter the jira issue belongs to")

In [79]:
qa = dspy.ChainOfThought(GenerateSummary)
response = qa(question=question)

In [80]:
response

Prediction(
    rationale='produce the answer. We need to identify the quarter the issue belongs to based on the provided labels. The label "PI24Q2Prioritized" indicates that the issue is prioritized for the second quarter of the year 2024.',
    answer='Q2 2024'
)

In [83]:
dataset = [dspy.Example( question = x[0], answer= str(x[2])).with_inputs("question") for x in data]
trainset = dataset[1:7]
testset = dataset[0:1] + dataset[7:]  #manually balancing lable types

In [84]:
class Flow(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(GenerateSummary)
    
    def forward(self, question):
        prediction = self.generate_answer(question=question)
        return dspy.Prediction(context=question, answer=prediction.answer)

In [85]:
from dspy.teleprompt import BootstrapFewShot

In [86]:
def validate_context_and_answer(example, pred, trace=None):
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_PM

In [87]:
optimizer = BootstrapFewShot(metric=validate_context_and_answer)

In [88]:
compiled_flow = optimizer.compile(Flow(), trainset=trainset)

100%|██████████| 6/6 [00:00<00:00, 48.71it/s]

Bootstrapped 0 full traces after 6 examples in round 0.


In [89]:
updated_response = compiled_flow(question="sumasummarize the jira issue:" + testset[2].question)
print(updated_response.answer)

Q3
